# Timeseries generation with HYP3_ISCE2

Single Bust for now Submit 2 interferograms as 'batch job'
https://hyp3-docs.asf.alaska.edu/guides/burst_insar_product_guide/

In [1]:
import hyp3_sdk as sdk
import asf_search as asf
import geopandas as gpd
import pandas as pd
import os
import copy
from folium import plugins

In [2]:
asf.__version__

'6.7.1'

In [3]:
hyp3 = sdk.HyP3() # 'https://hyp3-api.asf.alaska.edu'
print('User:{user_id} Quota:{quota}'.format(**hyp3.my_info()))

User:scottyhq Quota:{'max_jobs_per_month': 1000, 'remaining': 516}


## Search for acquisitions

In [4]:
# https://geojson.io/#new&map=15.23/47.654452/-122.303174
gfa = gpd.GeoDataFrame.from_features( {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          -122.29994319751066,
          47.657472535245574
        ],
        "type": "Point"
      }
    }
  ]
},
 crs=4326   
)

In [5]:
results = asf.search(
                    platform=[asf.PLATFORM.SENTINEL1],
                    processingLevel=asf.BURST,
                    beamMode=asf.BEAMMODE.IW,
                    #relativeOrbit=relorb,
                    polarization='VV',
                    flightDirection='ASCENDING',
                    intersectsWith=str(gfa.geometry.values[0]),
                    start='2019-06-01',
                    end='2023-09-01',
                    #season=[180,270], #first,last day of year June~30*6=180
                    #operaBurstID=['T064_135602_IW1'],
                    )
print(len(results))

gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)

244


In [6]:
gf.iloc[0]

geometry           POLYGON ((-123.281634 47.700793, -123.231493 4...
beamModeType                                                      IW
browse                                                          None
bytes                                                      160556256
centerLat                                                  47.680165
centerLon                                                 -122.65681
faradayRotation                                                 None
fileID                   S1_292399_IW2_20230829T020249_VV_CF75-BURST
flightDirection                                            ASCENDING
groupID                                S1A_IWDV_0152_0159_050084_137
granuleType                                                     None
insarStackId                                                    None
md5sum                                                          None
offNadirAngle                                                   None
orbit                             

In [7]:
gf['burstId'] = gf.burst.str['fullBurstID']
gf.groupby('burstId').sceneName.count()

burstId
064_135602_IW1    122
137_292399_IW2    122
Name: sceneName, dtype: int64

In [8]:
# Pick a single burst ID to work with
burst = '064_135602_IW1'

In [9]:
# Look at timeline of acquisition
gfb = gf[gf.burstId == burst]
gfb.loc[:,'datetime'] = pd.to_datetime(gfb.startTime)

/home/jovyan/.local/envs/hyp3-isce2/lib/python3.11/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [10]:
m = gf.iloc[[0]].explore()
minimap = plugins.MiniMap(width=200, height=100)
m.add_child(minimap)